In [60]:
from langchain_community.document_loaders import AmazonTextractPDFLoader
from langchain_community.document_loaders import PDFPlumberLoader
from langchain.prompts import PromptTemplate
from langchain.llms.bedrock import Bedrock
from langchain.chains import LLMChain
import boto3
import botocore

config = botocore.config.Config(
    read_timeout=900,
    connect_timeout=900,
    retries={"max_attempts": 3}
)

bedrock_client = boto3.client(
    service_name="bedrock-runtime",
    region_name="us-east-1",
    config=config,
)

textract_client = boto3.client("textract", region_name="us-west-2")
textract_features=["LAYOUT"]
file_path = "s3://my-s3-doc-loader/Sample6.pdf"
loader = AmazonTextractPDFLoader(file_path,textract_features,client=textract_client)
docs = loader.load()

In [61]:
all_page_content = ""
for doc in docs:
    all_page_content += doc.page_content
print(all_page_content)

Anna pakker alt fra
faerosk laks til snegle
fra Hundested i sine
fiskekasser.


Fra mode- designer til fiskehandler
Det var sindssygt
hardt, men jeg vidste,
det kunne lykkes
Ifort et par sorte gummistov-
Med et bevis fra Kolding Designskole
Pa trods af sine betenkelighe-
ler og et hvidt forklade smaek-
der blev Anna pa skolebenken
ker Anna Kirstine Borg et par
under armen flyttede Anna til Kobenhavn
i alle fem ar. Da hun blev faer-
store torskeloins op pa bordet
med drommen om at arbejde som
diguddannet, gik turen til Ko-
i sin lille hvide kolevogn. Bag
modedesigner. I mange ar designede hun alt
benhavn, hvor hun flyttede ind
hende ligger kassevis af frisk
i et kollektiv. I omkring seks ar
fisk og venter pa at blive skãret
fra hospitalsklovnekostumer til rollatortasker,
designede og syede hun alt fra
ud og frosset ned.
mens en helt anden drom begyndte at fylde
teaterkostumer og brudekjoler
Anna har vaeret tidligt oppe i
til hospitalsklovnenes kitler og
morges for at hente frisk baere-


In [53]:
template = """
Extract all the text datas in the given text documents and print all the text as a result in the formatted way as per the given document

Instruction:
- Extract all the datas ,Ensure not to skip any datas
- The given text document consists of two column,so you must give the result as the single column so that i can easy to be convert as a pdf
- Strictly print the paragraph in the single line to convert the text in the document as single column

{datas}
"""
qa_prompt = PromptTemplate(template=template, input_variables=["datas"])
llm = Bedrock(model_id="anthropic.claude-v2:1",client=bedrock_client,model_kwargs = {"temperature":1e-10,"max_tokens_to_sample": 40000})
llm_chain = LLMChain(prompt=qa_prompt, llm=llm, verbose= False)
result = llm_chain.run(datas= docs)

In [55]:
text_to_remove_1 = "Here is the extracted text data from the given document formatted in a single column:"
text_to_remove_2 = "Here is the extracted text data printed in a formatted single column:"
text_remove = "Here is the extracted text data from the document formatted in a single column:"
text_to_remove_4 = "Here is the extracted text data from the given documents formatted as a single column:"
text_to_remove_5 = "Here is the extracted text from the document formatted in a single column:"
text_to_remove_6="Here is the extracted text data from the given document formatted as a single column:"

cleaned_paragraph = result.replace(text_to_remove_1, "").replace(text_to_remove_2, "").replace(text_remove, "").replace(text_to_remove_4, "").replace(text_to_remove_5, "").replace(text_to_remove_6, "")


In [56]:
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Paragraph
from reportlab.lib.styles import getSampleStyleSheet

# Create a PDF document
doc = SimpleDocTemplate(r"C:\Users\Lenovo\Documents\Project-vs code\Amazon Transcribe\2-Cloumn page\output_2column\Output_sample8.pdf", pagesize=letter)

# Get the styles for the document
styles = getSampleStyleSheet()

# Create a list of paragraphs from the text
paragraphs = []
for line in cleaned_paragraph.split("\n"):
    paragraph = Paragraph(line, styles["BodyText"])
    paragraphs.append(paragraph)

# Build the PDF document
doc.build(paragraphs)